# Apprentissage sur les images

Entraine le modèle sur des images et pas sur les vidéos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 8
%config InlineBackend.figure_format = 'retina'

## Importation
Les images utilisée pour entrainer le réseau convolutionnel de VGG ont été normalisées.
Il faut appliquer la même transformation à nos images pour conserver les propriétés non-linéaires du réseau de convolution.

In [2]:
preprocess = ImageDataGenerator(
    preprocessing_function=keras.applications.vgg16.preprocess_input
)

train_dataset = preprocess.flow_from_directory(
    directory='DB/train/',
    classes=None,
    class_mode='categorical',
    batch_size=32,
    target_size=(227, 227)
)

test_dataset = preprocess.flow_from_directory(
    directory="DB/test/",
    classes=None,
    class_mode="categorical",
    batch_size=32,
    target_size=(227, 227)
)

Found 4736 images belonging to 5 classes.
Found 3568 images belonging to 5 classes.


## Modèle de classification
Les images sont
- peu nombreuses
- parfois floues
- une main recouvre en partie les objets
- d'autres objets sont visibles en arrière plan

Le modèle pré-entrainé VGG permet de traiter efficacement les images.
La classification est réduite aux seules 5 classes cibles, le modèle VGG-imagenet complet contient trop de classes et ne parvient pas à distinguer nos 5 objets parmis tous les objets possibles.

In [3]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Input, Flatten, Dense, Dropout

vgg = VGG16(include_top=False, weights="imagenet")
vgg.trainable = False

dense = Sequential([
    Flatten(),
    Dense(512, activation="elu"),
    Dropout(0.1),
    Dense(256),
    Dense(5, activation="softmax")
], name="dense")

modele = Sequential([
    Input((227, 227, 3)),
    vgg,
    dense
], name="complet")

dense.summary()
modele.summary()

Model: "dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
Total params: 12,978,181
Trainable params: 12,978,181
Non-trainable params: 0
_________________________________________________________________


In [4]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy

optimiser = Adam()
modele.compile(optimizer=optimiser, loss=categorical_crossentropy, metrics=["accuracy", "categorical_accuracy"])
historique = modele.fit(train_dataset,
                        epochs=2
                       #validation_data=test_dataset
                       )
dense.save("modele-VGG16")

Epoch 1/2
148/148 [==============================] - 637s 4s/step - loss: 3.1824 - accuracy: 0.9576
Epoch 2/2
148/148 [==============================] - 615s 4s/step - loss: 0.0140 - accuracy: 0.9975
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: modele-VGG16/assets
